In [ ]:
from google.colab import files
import matplotlib.pyplot as plt
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
from tensorflow import keras
import numpy as np
import seaborn as sns
import tensorflow as tf
import time
import torch
from torch import nn
from torch import optim

random_state=30

try:
    from nflows.flows.base import Flow
    from nflows.distributions.normal import ConditionalDiagonalNormal
    from nflows.transforms.base import CompositeTransform
    from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
    from nflows.transforms.permutations import ReversePermutation
    from nflows.nn.nets import ResidualNet
except:
    !pip install nflows
    from nflows.flows.base import Flow
    from nflows.distributions.normal import ConditionalDiagonalNormal
    from nflows.transforms.base import CompositeTransform
    from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
    from nflows.transforms.permutations import ReversePermutation
    from nflows.nn.nets import ResidualNet

In [ ]:
x, y = datasets.make_moons(128, noise=.1, random_state=random_state)
plt.scatter(x[:, 0], x[:, 1], c=y);

In [ ]:
# Make NF
num_layers = 5
base_dist = ConditionalDiagonalNormal(shape=[2], 
                                      context_encoder=nn.Linear(1, 4))

transforms = []
for _ in range(num_layers):
    transforms.append(ReversePermutation(features=2))
    transforms.append(MaskedAffineAutoregressiveTransform(features=2, 
                                                          hidden_features=4, 
                                                          context_features=1))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist)
optimizer = optim.Adam(flow.parameters())

In [ ]:
# Train NF
num_iter = 5000
x, y = datasets.make_moons(1000, noise=.1)
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)
start = time.time()
for i in tqdm(range(num_iter)):
    
    optimizer.zero_grad()
    loss = -flow.log_prob(inputs=x, context=y).mean()
    loss.backward()
    optimizer.step()
    
    if (i + 1) % 500 == 0:
        fig, ax = plt.subplots(1, 2, figsize=(15,8))
        xline = torch.linspace(-1.5, 2.5, 100)
        yline = torch.linspace(-.75, 1.25, 100)
        xgrid, ygrid = torch.meshgrid(xline, yline)
        xyinput = torch.cat([xgrid.reshape(-1, 1), ygrid.reshape(-1, 1)], dim=1)

        with torch.no_grad():
            zgrid0 = flow.log_prob(xyinput, torch.zeros(10000, 1)).exp().reshape(100, 100)
            zgrid1 = flow.log_prob(xyinput, torch.ones(10000, 1)).exp().reshape(100, 100)
            T=2
            for _ in tqdm(range(T-1)):
                zgrid0 += flow.log_prob(xyinput, torch.zeros(10000, 1)).exp().reshape(100, 100)
                zgrid1 += flow.log_prob(xyinput, torch.ones(10000, 1)).exp().reshape(100, 100)
                zgrid0_mean = zgrid0.numpy()/T
                zgrid1_mean = zgrid1.numpy()/T

        ax[0].contourf(xgrid.numpy(), ygrid.numpy(), zgrid0_mean)
        ax[1].contourf(xgrid.numpy(), ygrid.numpy(), zgrid1_mean)
        plt.title('iteration {}'.format(i + 1))
        # if i+1 == num_iter:
            # x_ = x.numpy()
            # y_ = y.squeeze().numpy()
            # x0 = x_[y_==0]
            # x1 = x_[y_==1]
            # ax[0].scatter(x0[:, 0], x0[:, 1], color='red');
            # ax[1].scatter(x1[:, 0], x1[:, 1], color='red');

            # plt.savefig('nf.png')
            # files.download('nf.png')
            # pass
        plt.show()
        print('')
print(f"Time to train: {time.time()-start} seconds")

In [ ]:
import seaborn as sns
x, y = datasets.make_moons(1000, noise=.1, random_state=random_state)
plt.figure(figsize=(10,8))

sns.scatterplot(x[:, 0], x[:, 1], c=y)
plt.savefig('moons.png')
# files.download('moons.png')

In [ ]:
x_ = x
y_ = y
x0 = x_[y_==0]
x1 = x_[y_==1]

In [ ]:
num_examples = 300
zero = flow.sample(num_examples, torch.zeros(100, 1)).detach().numpy()
one = flow.sample(num_examples, torch.ones(100, 1)).detach().numpy()

plt.figure(figsize=(10,8))
plt.scatter(zero[0][:, 0], zero[0][:, 1], color='green')
plt.scatter(one[0][:, 0], one[0][:,1], color='gray')
plt.show()

In [ ]:
plt.figure(figsize=(15,8))
plt.contourf(xgrid.numpy(), ygrid.numpy(), (zgrid0_mean+zgrid1_mean))
num_pts = 500
plt.scatter(x[0:num_pts,0], x[0:num_pts,1], s=20, c=y[0:num_pts], cmap='flag')
# plt.title("Moon Density Estimation with Normalizing Flow")
plt.savefig('moons_density.png')
# files.download('moons_density.png')

# TODO: Figure out colors and do the same (maybe X's for incorrect) on Neural Net
plt.tight_layout()
plt.show()

In [ ]:
start = time.time()

# (X_train, y_train), (X_test, y_test) = keras.datasets.boston_housing.load_data()
x, y = datasets.make_moons(10000, noise=.1)
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size = 0.8)

dropout_prob = 0.2

# Create NN with Dropout layers after every connected layer
model = keras.models.Sequential()
model.add(keras.layers.Dropout(dropout_prob))
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dropout(dropout_prob))
model.add(keras.layers.Dense(64, activation="relu"))
model.add(keras.layers.Dropout(dropout_prob))
model.add(keras.layers.Dense(1, activation="sigmoid"))

optimizer = keras.optimizers.Adam()
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_split=0.1, verbose=False)
print(f"{time.time() - start}")
mse, accuracy = model.evaluate(X_test, y_test)
print(f'binary_crossentropy: {round(mse, 3)}, {accuracy}')

In [ ]:
num_iterations = 1000
predictions = []
for _ in tqdm(range(num_iterations)):
    predictions.append(model(X_test, training=True))
mean, std = np.mean(np.array(predictions), axis=0), np.std(np.array(predictions), axis=0)

In [ ]:
T = 50
predictions = []
y_pred = model(X_test, training=True)
for _ in tqdm(range(T-1)):
    y_pred += model(X_test, training=True)
y_pred = np.array(y_pred)
y_pred = y_pred/T

P_foreground = y_pred
P_background = 1-P_foreground

UQ = -(P_foreground*np.log(P_foreground)+P_background*np.log(P_background))

In [ ]:
y_pred

In [ ]:
threshold=0.5
# y_pred_class = y_pred
y_pred_class = model.predict(X_test)
y_pred_class[y_pred<threshold]=0
y_pred_class[y_pred>=threshold]=1

In [ ]:
sum(y_test == y_pred_class.squeeze()) / 2000.

In [ ]:
markers = ['o' if i == True else 'x' for i in y_test == y_pred_class.squeeze()]
acc = y_test == y_pred_class.squeeze()
size = (-80 * acc) + 100

In [ ]:
plt.figure(figsize=(15,8))
for i in range(len(X_test)):
    plt.scatter(X_test[i,0], X_test[i,1], c=y_pred_class[i], cmap='flag', vmin=0, vmax=1, marker=markers[i], s=size[i])
# plt.colorbar()
# plt.title("Neural Network Output")

plt.savefig("mean.png")
# files.download("mean.png") 
# plt.legend(c)
plt.show()

In [ ]:
def plot_decision_boundary(X, y, model, steps=1000, cmap='Paired'):
    """
    Function to plot the decision boundary and data points of a model.
    Data points are colored based on their actual label.
    """
    cmap = plt.get_cmap(cmap)

    # Define region of interest by data limits
    xmin, xmax = X[:,0].min() - 1, X[:,0].max() + 1
    ymin, ymax = X[:,1].min() - 1, X[:,1].max() + 1
    steps = 1000
    x_span = np.linspace(xmin, xmax, steps)
    y_span = np.linspace(ymin, ymax, steps)
    xx, yy = np.meshgrid(x_span, y_span)

    # Make predictions across region of interest
    labels = model.predict(np.c_[xx.ravel(), yy.ravel()])
    # Plot decision boundary in region of interest
    z = labels.reshape(xx.shape)

    fig, ax = plt.subplots(figsize=(15,8))
    ax.contourf(xx, yy, z, cmap=cmap, alpha=0.5)

    # Get predicted labels on training data and plot
    train_labels = model.predict(X)
    predicted_labels = train_labels > 0.5
    ax.scatter(X[:,0], X[:,1], c=predicted_labels, cmap=cmap, lw=0)

    return fig, ax

plot_decision_boundary(X_test, y_test, model, cmap='RdBu')
plt.show()